# Connecting to HDInsight from Compute Instance
Leveraging [JayDeBeAPI python package](https://github.com/baztian/jaydebeapi). 

### Install JDBC Driver and jaydebeapi package from PyPI

In [ ]:
# Open terminal and execute following command prior to executing cell
# pip install jaydebeapi

!mkdir jdbcdriver
!wget -O './jdbcdriver/hive-jdbc-uber-2.6.3.0-235.jar' https://github.com/timveil/hive-jdbc-uber-jar/releases/download/v1.8-2.6.3/hive-jdbc-uber-2.6.3.0-235.jar

### Connect to AML Workspace and retrieve pointer to default Azure Key Vault
Retrieve HDI password from key vault. Note: this can be set using the syntax `kv.set_secret('hdipassword', 'XXXXXXXX')`.

In [ ]:
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset, Model

# Connect to AML Workspace
ws = Workspace.from_config()

# Get default key vault
kv = ws.get_default_keyvault()

### Retrieve data from HDInsight

In [ ]:
import jaydebeapi
import pandas as pd

driver = 'org.apache.hive.jdbc.HiveDriver'
driver_path = './jdbcdriver/hive-jdbc-uber-2.6.3.0-235.jar'
username = 'admin'
password = kv.get_secret('hdipassword')
server = kv.get_secret('hdiserver')

conn = jaydebeapi.connect(driver,
       f"jdbc:hive2://{server}:443/;ssl=true;transportMode=http;httpPath=/hive2", 
       [username, password],
       driver_path)

curs = conn.cursor()
curs.execute('select * from hivesampletable limit 25')
rows = curs.fetchall()   
df = pd.DataFrame(rows, columns=[column[0].replace('hivesampletable.','') for column in curs.description])
df

# Connect to HDInsight from Compute Cluster (AML Pipeline)

### Import required packages

In [ ]:
# Import required packages
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute, DataFactoryCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig
from azureml.data.datapath import DataPath
from azureml.data.data_reference import DataReference
from azureml.data.sql_data_reference import SqlDataReference
from azureml.pipeline.steps import DataTransferStep
import logging

### Connect to AML workspace, create compute cluster, and retrieve pointer to default datastore

In [ ]:
# Connect to AML Workspace
ws = Workspace.from_config()

#Select AML Compute Cluster
cpu_cluster_name = 'cpucluster'

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found an existing cluster, using it instead.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',
                                                           min_nodes=0,
                                                           max_nodes=1)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)
    
#Get default datastore
default_ds = ws.get_default_datastore()

### Define RunConfiguration

In [ ]:
dockerfile = r"""FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
RUN mkdir -p /usr/share/man/man1
RUN apt-get update && \
    DEBIAN_FRONTEND=noninteractive \
    apt-get -y install default-jre-headless && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*
RUN mkdir jdbcdriver
RUN wget -O './jdbcdriver/hive-jdbc-uber-2.6.3.0-235.jar' https://github.com/timveil/hive-jdbc-uber-jar/releases/download/v1.8-2.6.3/hive-jdbc-uber-2.6.3.0-235.jar
ENV CLASSPATH="/jdbcdriver/hive-jdbc-uber-2.6.3.0-235.jar:${CLASSPATH}"
"""

run_config = RunConfiguration()
run_config.docker.use_docker = True
run_config.environment = Environment(name='hdi_env')
run_config.environment.docker.base_image = None
run_config.environment.docker.base_dockerfile = dockerfile
run_config.environment.python.conda_dependencies = CondaDependencies.create()
run_config.environment.python.conda_dependencies.set_pip_requirements([
    'pandas==0.25.3',
    'numpy==1.19.2',
    'azureml-defaults==1.40.0',
    'jaydebeapi==1.2.3'
])
run_config.environment.python.conda_dependencies.set_python_version('3.8.10')
#Register environment for reuse 
run_config.environment.register(ws)

### Define Output Datasets

In [ ]:
hdi_data = OutputFileDatasetConfig(name='HDI_Data', destination=(default_ds, 'hdi_data/{run-id}')).read_delimited_files().register_on_complete(name='HDI_Data')

### Define Pipeline Steps

In [ ]:
# Get raw data from AML-linked datastore
# Register tabular dataset after retrieval
get_data_step = PythonScriptStep(
    name='Get Data from Blob Storage',
    script_name='query_hdinsight.py',
    arguments =['--hdi_data', hdi_data],
    outputs=[hdi_data],
    compute_target=cpu_cluster,
    source_directory='./pipeline_step_scripts',
    allow_reuse=False,
    runconfig=run_config
)

### Create AML Pipeline

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[get_data_step])

### Submit Pipeline run

In [ ]:
experiment = Experiment(ws, 'hdi-query-testing')
run = experiment.submit(pipeline)
run.wait_for_completion(show_output=True)